In [2]:
import sys, os
import numpy as np

import PIL
import matplotlib.pyplot as plt

import pandas as pd

In [6]:
orig_fn = '/user_data/mmhender/stimuli/featsynth/images_comb64_alllayers_50steps/butterfly/ex03/orig.png'

print(orig_fn)

os.chdir('/user_data/mmhender/featsynth/texture_synthesis/code/')
sys.path.append('/user_data/mmhender/featsynth/texture_synthesis/code/')
import synthesize_textures

out_dir = 'tmp'

print(out_dir)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

rndseed = 868214
    
synthesize_textures.make_textures_oneimage(orig_fn, 
                                           out_dir,
                                           rndseed=rndseed,
                                           n_steps = 50,
                                           all_layers = True, \
                                           save_loss=True)

/user_data/mmhender/stimuli/featsynth/images_comb64_alllayers_50steps/butterfly/ex03/orig.png
tmp
saving image to tmp/orig.png
took 0.07622 s to preproc and save orig image
took 0.00963 s to preproc image for synthesis
0 4
making texture for layers:
['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1']
matching layers:
['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1']
seed is: 868215
step: 1, loss: 943268288.0 (0.24s)
step: 2, loss: 66299340.0 (0.22s)
step: 3, loss: 17914342.0 (0.28s)
step: 4, loss: 6544468.0 (0.31s)
step: 5, loss: 3352485.5 (0.29s)
step: 6, loss: 1779004.25 (0.29s)
step: 7, loss: 1135616.25 (0.29s)
step: 8, loss: 779555.75 (0.30s)
step: 9, loss: 558165.875 (0.31s)
step: 10, loss: 441121.0 (0.29s)
step: 11, loss: 349152.09375 (0.31s)
step: 12, loss: 289199.21875 (0.31s)
step: 13, loss: 252473.8125 (0.30s)
step: 14, loss: 219866.265625 (0.29s)
step: 15, loss: 194745.84375 (0.29s)
step: 16, loss: 175928.96875 (0.29s)
step: 17, loss: 157114.859375 (0.29s)
step: 18, lo

AssertionError: 

In [24]:
target_image_filename = orig_fn
out_dir = out_dir
rndseed = 868218
layers_do = ['pool1','pool2','pool3','pool4']
n_steps = 50
all_layers = True
save_loss=True

In [25]:
import os
import sys
import numpy as np
import torch
import pandas as pd
import PIL
import time

# this is where the saved model files is located
texture_synth_root = os.path.dirname(os.getcwd())

import utilities
import model_spatial
import optimize

import model_spatial_combineimages
import optimize_combineimages

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [26]:
model_path = os.path.join(texture_synth_root, 'models','VGG19_normalized_avg_pool_pytorch')

class a:
    def __init__():
        return a
    
args = a
args.lr = 1.0
args.max_iter = 20
args.checkpoint_every = 1
args.n_steps = n_steps
args.do_sqrt = True
args.rndseed = rndseed


st = time.time()
target_image_tosave = utilities.preprocess_image_tosave(
    utilities.load_image(target_image_filename)
)
filename_save = os.path.join(out_dir, 'orig.png')
print('saving image to %s'%filename_save)
target_image_tosave.save(filename_save)
elapsed = time.time() - st
print('took %.5f s to preproc and save orig image'%elapsed)

st = time.time()
# load target image to use in synthesis (different preproc than above)
target_image = utilities.preprocess_image(
    utilities.load_image(target_image_filename)
)
elapsed = time.time() - st
print('took %.5f s to preproc image for synthesis'%elapsed)


saving image to tmp/orig.png
took 0.08147 s to preproc and save orig image
took 0.00757 s to preproc image for synthesis


In [27]:

if all_layers:
    # for this we will consider every layer in the model
    layers_all = ['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1', 'conv2_1',
       'relu2_1', 'conv2_2', 'relu2_2', 'pool2', 'conv3_1', 'relu3_1',
       'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4',
       'pool3', 'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
       'relu4_3', 'conv4_4', 'relu4_4', 'pool4', 'conv5_1', 'relu5_1',
       'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_4', 'relu5_4',
       'pool5']
else:
    # not considering every feature map but just a key subset.
    # this is like the "~177k parameters" version in Gatys 2015.
    # doesn't seem  to make huge difference.
    layers_all = ['relu1_1', 'pool1','pool2','pool3','pool4']
    
# layers_all

In [32]:
# loop over layers - making a different texture for each layer
# each will match the statistics of that layer and the ones before.

# ll = 2;
# for ll in range(len(layers_do)):
# 
curr_layer = layers_do[ll]

# current layer has to match one of the names in layers_all
index = np.where(curr_layer==np.array(layers_all))[0][0]
print(ll, index)

# which layers to match now? grab all layers up to and including this one.
layers_match = layers_all[0:index+1]

print('making texture for layers:')
print(layers_match)
sys.stdout.flush()

st = time.time()
net = model_spatial.Model(model_path, device, target_image, \
                          important_layers = layers_match, \
                          spatial_weights_list = None, 
                          layer_weights = [1e09 for l in layers_match], \
                          do_sqrt = args.do_sqrt)

# synthesize
# args.rndseed += 1
# args.rndseed = 354642
optimizer = optimize.Optimizer(net, args)
result = optimizer.optimize()
elapsed = time.time() - st
print('took %.5f s to run synthesis'%elapsed)


2 18
making texture for layers:
['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1', 'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2', 'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3']
matching layers:
['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1', 'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2', 'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3']
seed is: 868218
step: 1, loss: 20017143808.0 (0.58s)
step: 2, loss: 910743488.0 (0.51s)
step: 3, loss: 226351264.0 (0.60s)
step: 4, loss: 103938144.0 (0.57s)
step: 5, loss: 58800744.0 (0.57s)
step: 6, loss: 37550212.0 (0.64s)
step: 7, loss: 25447008.0 (0.57s)
step: 8, loss: 19351316.0 (0.59s)
step: 9, loss: 14766562.0 (0.59s)
step: 10, loss: 12139928.0 (0.55s)
step: 11, loss: 10165132.0 (0.57s)
step: 12, loss: 8677336.0 (0.55s)
step: 13, loss: 7648697.0 (0.55s)
step: 14, loss: 6768252.0 (0.59s)
step: 15, loss: 6094213.0 (0.57s)
step: 16,

In [31]:
net

In [68]:
orig_stim_path = '/user_data/mmhender/stimuli/featsynth/images_comb64_orig'

image_list_filename = os.path.join(orig_stim_path, 'images_comb64_list.csv')
print(image_list_filename)
image_list = pd.read_csv(image_list_filename, index_col=0)


/user_data/mmhender/stimuli/featsynth/images_comb64_orig/images_comb64_list.csv


In [69]:
image_list

,super_name,super_index,basic_name,basic_index,image_type,exemplar_number,image_filename,orig_set,exemplar_number_orig,image_filename_new,...,seed40,seed41,seed42,seed43,seed44,seed45,seed46,seed47,seed48,seed49
0,insect,0,beetle,0,orig,0,/user_data/mmhender/stimuli/things/Images/beet...,things,0,/user_data/mmhender/stimuli/featsynth/images_c...,...,334959,307683,453072,369174,606363,493089,542544,262467,327855,528690
1,insect,0,beetle,0,orig,1,/user_data/mmhender/stimuli/things/Images/beet...,things,1,/user_data/mmhender/stimuli/featsynth/images_c...,...,440817,308160,587109,335607,256995,283008,276285,477258,353163,383247
2,insect,0,beetle,0,orig,2,/user_data/mmhender/stimuli/things/Images/beet...,things,2,/user_data/mmhender/stimuli/featsynth/images_c...,...,271029,306723,348399,615669,358131,499167,519396,535509,435081,235194
3,insect,0,beetle,0,orig,3,/user_data/mmhender/stimuli/things/Images/beet...,things,3,/user_data/mmhender/stimuli/featsynth/images_c...,...,303021,515904,389361,609318,406620,319197,551136,366516,489915,468900
4,insect,0,beetle,0,orig,4,/user_data/mmhender/stimuli/things/Images/beet...,things,4,/user_data/mmhender/stimuli/featsynth/images_c...,...,440301,283911,390744,530208,373803,481341,554622,409614,298134,279336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,vehicle,7,canoe,63,orig,35,/lab_data/tarrlab/common/datasets/Ecoset/train...,ecoset,132,/user_data/mmhender/stimuli/featsynth/images_c...,...,326193,471669,596430,474618,333738,468603,296679,579216,330663,506937
2556,vehicle,7,canoe,63,orig,36,/lab_data/tarrlab/common/datasets/Ecoset/train...,ecoset,224,/user_data/mmhender/stimuli/featsynth/images_c...,...,286245,362259,438552,457410,594816,574911,288531,249888,316698,539784
2557,vehicle,7,canoe,63,orig,37,/lab_data/tarrlab/common/datasets/Ecoset/train...,ecoset,74,/user_data/mmhender/stimuli/featsynth/images_c...,...,338739,361833,287931,340380,530670,586029,578160,329451,247617,547821
2558,vehicle,7,canoe,63,orig,38,/lab_data/tarrlab/common/datasets/Ecoset/train...,ecoset,146,/user_data/mmhender/stimuli/featsynth/images_c...,...,309381,263814,611979,542409,529008,255900,293259,372924,321192,483954


In [77]:
rndseed = 5
hasattr(rndseed, '__len__')

False

In [60]:
image_list.shape

(2560, 10)

In [67]:
def get_seeds():
    
    n_categ = 64
    n_ex = 40
    n_imtypes = 5
    n_each = 10
    
    n_seeds = n_categ * n_ex * n_imtypes * n_each
    
    s = 234534
    allseeds = np.array([s + i*3 for i in range(n_seeds)])
    
    np.random.seed(353545)
    
    allseeds = allseeds[np.random.permutation(n_seeds)]
    
    allseeds = np.reshape(allseeds, [n_categ * n_ex, n_imtypes*n_each])

    return allseeds

In [62]:
allseeds

array([[258327, 249987, 585903, ..., 262467, 327855, 528690],
       [315219, 447771, 489387, ..., 477258, 353163, 383247],
       [495813, 260649, 529785, ..., 535509, 435081, 235194],
       ...,
       [371184, 500331, 257925, ..., 329451, 247617, 547821],
       [247056, 491988, 341937, ..., 372924, 321192, 483954],
       [348561, 435051, 577512, ..., 286920, 262026, 358014]])

In [64]:
for x in range(allseeds.shape[1]):

    image_list['seed%d'%(x)] = allseeds[:,x]

In [66]:
image_list.keys()

Index(['super_name', 'super_index', 'basic_name', 'basic_index', 'image_type',
       'exemplar_number', 'image_filename', 'orig_set', 'exemplar_number_orig',
       'image_filename_new', 'seed0', 'seed1', 'seed2', 'seed3', 'seed4',
       'seed5', 'seed6', 'seed7', 'seed8', 'seed9', 'seed10', 'seed11',
       'seed12', 'seed13', 'seed14', 'seed15', 'seed16', 'seed17', 'seed18',
       'seed19', 'seed20', 'seed21', 'seed22', 'seed23', 'seed24', 'seed25',
       'seed26', 'seed27', 'seed28', 'seed29', 'seed30', 'seed31', 'seed32',
       'seed33', 'seed34', 'seed35', 'seed36', 'seed37', 'seed38', 'seed39',
       'seed40', 'seed41', 'seed42', 'seed43', 'seed44', 'seed45', 'seed46',
       'seed47', 'seed48', 'seed49'],
      dtype='object')

In [63]:
allseeds.shape

(2560, 50)

In [46]:
allseeds[-1]

490530

In [37]:
import time, datetime



In [38]:
t = time.time()

In [40]:
t.datetime.strftime('%H')

AttributeError: 'float' object has no attribute 'datetime'